##### Exercise 1 Hello World

1. Write an MPI program displaying the number of processes used for the execution and the rank of each process.
2. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
Hello from the rank 2 process
Hello from the rank 0 process
Hello from the rank 3 process
Hello from the rank 1 process
Parallel execution of hello_world with 4 process
```
*Note that the output order maybe different*

# NB : Je n'ai que deux processeurs. Ainsi, je n'ai pas pu réaliser les graphiques afin de voir la scalabilité

In [1]:
%%file hello.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
print("Hello from the rank ", RANK," process", "\n")
if RANK==1:
    print("Parallel execution of hello_world with ",SIZE," process")

Overwriting hello.py


In [2]:
# enter command for compile and run the program
!mpirun -n 2 python3 hello.py 

Hello from the rank  0  process 

Hello from the rank  1  process 

Parallel execution of hello_world with  2  process


## Exercise 2 Sharing Data 

A common need is for one process to get data from the user, either by reading from the terminal or command line arguments, and then to distribute this information to all other processors.

Write a program that reads an integer value from the terminal and distributes the value to all of the MPI processes. Each process should print out its rank and the value it received. Values should be read until a negative integer is given as input.

You may want to use these MPI routines in your solution:
`Get_rank` `Bcast` 

**Output Example**
```shell
10
Process 0 got 10
Process 1 got 10
```

In [3]:
 %%file sharing.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

recv = 5
send =1
# COMM.Barrier()
while recv > 0:
    if RANK == 0 :
      send = int(input('Enter a number : ')) 
        # send = 10
      print(send)
    else: 
      send = None

    recv = COMM.bcast(send, root=0)
    print("Process ",RANK, " got ", recv)


Overwriting sharing.py


In [4]:
# enter command for compile and run the program
!mpirun -n 2 python3 sharing.py 

Enter a number : ^C


## Exercise 3 Sending in a ring (broadcast by ring)

Write a program that takes data from process zero and sends it to all of the other processes by sending it in a ring. That is, process i should receive the data and send it to process i+1, until the last process is reached.
Assume that the data consists of a single integer. Process zero reads the data from the user.
![](../data/ring.gif)

You may want to use these MPI routines in your solution:
`Send` `Recv` 

In [4]:
 %%file sending.py
from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

tag = 8
# COMM.Barrier()
n = 2 #int(input("Enter the number of processor used :"))
if RANK == 0:
    send = 10
    recv = 10
    COMM.send(send, dest=RANK+1, tag=tag)
else:
    if n==2:
        recv = COMM.recv(source=0, tag=tag)
    else:
        for i in range(1,n):
            RANK = i
            recv = COMM.recv(source=RANK-1, tag=tag)
            COMM.send(recv, dest=RANK+1, tag=tag)
            
print("Process",RANK, " got ", recv)

Overwriting sending.py


In [5]:
# enter command for compile and run the program
!mpirun -n 2 python3 sending.py 

Process 0  got  10
Process 1  got  10


## Exercise 4 Matrix vector product

1. Use the `MatrixVectorMult.py` file to implement the MPI version of matrix vector multiplication.
2. Process 0 compares the result with the `dot` product.
3. Plot the scalability of your implementation. 

**Output Example**
```shell
CPU time of parallel multiplication using 2 processes is  174.923446
The error comparing to the dot product is : 1.4210854715202004e-14
```

In [6]:
 %%file MatrixVectorMult_V0.py
import numpy as np
from scipy.sparse import lil_matrix
from numpy.random import rand, seed
from numba import njit
from mpi4py import MPI
 # write your program here
''' This program compute parallel csc matrix vector multiplication using mpi '''

COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()

seed(42)

def matrixVectorMult(A, b, x):
    
    row, col = A.shape
    for i in range(row):
        a = A[i]
        for j in range(col):
            x[i] += a[j] * b[j]

    return 0

########################initialize matrix A and vector b ######################
#matrix sizes
SIZE = 1000
Local_size = SIZE//2

# counts = block of each proc
#counts = 

if RANK == 0:
    A = lil_matrix((SIZE, SIZE))
    A[0, :100] = rand(100)
    A[1, 100:200] = A[0, :100]
    A.setdiag(rand(SIZE))
    A = A.toarray()
    b = rand(SIZE)
else :
    A = None
    b = None
    
    
#########Send b to all procs and scatter A (each proc has its own local matrix#####
LocalMatrix = np.zeros((Local_size ,SIZE))
# Scatter the matrix A

b = COMM.bcast(b, root=0)

COMM.Scatter(A,LocalMatrix,root=0)

##################### Compute A*b locally #######################################
LocalX = np.zeros(Local_size)
#LocalMatrix = A
start = MPI.Wtime()
matrixVectorMult(LocalMatrix, b, LocalX)
stop = MPI.Wtime()
if RANK == 0:
    print("CPU time of parallel multiplication is ", (stop - start)*1000)

################## Gather te results ###########################################
# sendcouns = local size of result
#sendcounts = 
if RANK == 0: 
    X = np.zeros(SIZE)
else :
    X = None

#Gather the result into X
sendbuf = LocalX
COMM.Gather(sendbuf,X,root=0)

################## Print the results ###########################################

if RANK == 0 :
    X_ = A.dot(b)
    print("The result of A*b using dot is :", X_)
    print("\n")
    print("\n")
    print("The result of A*b using parallel version is :", X)

Overwriting MatrixVectorMult_V0.py


In [10]:
# enter command for compile and run the program
!mpirun -n 2 python3 MatrixVectorMult_V0.py 

CPU time of parallel multiplication is  390.248934
The result of A*b using dot is : [2.63781326e+01 2.13071154e+01 2.77823379e-01 4.71827620e-01
 9.02944984e-01 4.33507343e-02 1.62610958e-01 5.72887944e-01
 1.59248241e-01 1.18468930e-02 2.36388833e-01 3.61845979e-02
 2.08082711e-01 4.33939974e-01 3.75570382e-01 5.05522269e-01
 7.35254166e-02 1.63707925e-01 2.37063708e-01 6.98528680e-02
 7.17610888e-01 8.56350993e-01 2.74159578e-01 8.90888020e-02
 1.49352687e-01 2.35275473e-01 7.11564540e-02 3.51568172e-01
 2.59097922e-03 1.32668544e-01 3.01963515e-01 1.10137874e-01
 9.71463444e-03 7.43371895e-02 6.44251319e-01 2.46057170e-02
 4.41598164e-01 3.48097396e-01 1.74755506e-01 5.75691271e-01
 7.93711965e-01 8.75722009e-02 3.37142519e-01 1.70216477e-01
 7.60604645e-02 3.24099781e-02 4.86082462e-01 3.30989925e-01
 4.37868852e-02 2.41668463e-01 6.43381927e-01 2.00516495e-01
 1.01060038e-01 3.32896778e-01 6.09734586e-01 1.82199031e-01
 1.06604129e-01 6.70888449e-01 2.07182755e-01 2.12983497e-02
 

## Exercise 5 Calculation of π (Monte Carlo)

1. Use the `PiMonteCarlo.py` file to implement the calculation of PI using Monte Carlo.
2. Process 0 prints the result.
3. Plot the scalability of your implementation. 

In [11]:
 %%file PiMonteCarlo_V0.py
 # write your program here
import random 
import timeit
import numpy as np
from scipy.sparse import lil_matrix
from numpy.random import rand, seed
from numba import njit
from mpi4py import MPI
 # write your program here
''' This program compute parallel csc matrix vector multiplication using mpi '''

COMM = MPI.COMM_WORLD
nbOfproc = COMM.Get_size()
RANK = COMM.Get_rank()

INTERVAL= 1000

random.seed(42)  

def compute_points():
    
    random.seed(42)  
    
    circle_points= 0

    # Total Random numbers generated= possible x 
    # values* possible y values 
    for i in range(INTERVAL**2): 
      
        # Randomly generated x and y values from a 
        # uniform distribution 
        # Rannge of x and y values is -1 to 1 
                
        rand_x= random.uniform(-1, 1) 
        rand_y= random.uniform(-1, 1) 
      
        # Distance between (x, y) from the origin 
        origin_dist= rand_x**2 + rand_y**2
      
        # Checking if (x, y) lies inside the circle 
        if origin_dist<= 1: 
            circle_points+= 1
      
        # Estimating value of pi, 
        # pi= 4*(no. of points generated inside the  
        # circle)/ (no. of points generated inside the square) 
    
     
    
    return circle_points

start = timeit.default_timer()
circle_points = compute_points()
end = timeit.default_timer()

if RANK==0:
    pi = 4* circle_points/ INTERVAL**2 
    print('Process 0 \n')
    print("Circle points number :",circle_points)
    print("Final Estimation of Pi=", pi, "cpu time :",end-start) 

Overwriting PiMonteCarlo_V0.py


In [12]:
# enter command for compile and run the program
!mpirun -n 2 python3 PiMonteCarlo_V0.py

Process 0 

Circle points number : 785061
Final Estimation of Pi= 3.140244 cpu time : 0.9248099149990594
